<h1>Proton Therapy Optimization</h1>

## Naive modeling
The first idea that comes to mind is to make a straight forward mathematical translation of the oral problem description. That would be to set a vector of $i$ integer variables $x_{i}$, representing the number of proton fractions that should be performed to patient $i$. 

We would then symbolize our input matrix as $c_{ij}$: the benefit of offering $j$ proton fractions to patient $i$. While this is a very intuitive model it suffers from a very non linear object function. In the simplest case where we would like to optimize the sum or average BED, the object functions to be maximized would be:

\begin{equation*}
    \begin{aligned}
        \underset{x}{\text{max}}
            & \sum_{i} c_{ix_{i}}\cdot x_{i} 
    \end{aligned}     
\end{equation*}



## A better modeling idea

Instead we would prefer to go with a linear model. For that reason we could select another set of decision variables, that is a binary matrix $x_{ij}$ where a value of 1 denotes that $j$ fractions should be performed on patient $i$. Obviously we should constraint every rows sum to 1 (a patient will receive 0,1, ... **or** N fractions).

Our object function can now be expressed as elegandly as:
\begin{equation*}
    \begin{aligned}
            F(x)=  \sum_{ij} c_{ij}\cdot x_{ij} 
    \end{aligned}     
\end{equation*}


This is obviously as linear as it gets. Lets code it!

In [ ]:
# GUROBI must be installed and a licence file must be discoverable in one of the default locations
from gurobipy import *

# Get some additional dependencies
import pandas as pd
import numpy as np
import time

In [ ]:
# Read the dataset into a 2d array. benefit[i,j] is the benefit patient i will get from j fractions
data = pd.read_csv('data/PayoffMatrix.txt', delim_whitespace = True)
benefit = data.values

In [ ]:
class BEDPredictor(object):
    """
    A base implementation of BED matrix estimation. Given the real BED matrix and 
    a time cost per access, the predictor can output an estimation of the BED matrix along 
    with the time needed to produce it.
    
    EXTENSIONS: If you want to try another predictor (f.e non linear interpolation or series prediction)
    all you have to do is inherit this base and override the estimate() function. Make sure you
    obey its contract: it should return the predicted BED matrix as a 2D np.array. Everything else will 
    just work.
    """
    def __init__(self, BED):
        self._BED = BED
        self.num_patients, self.max_fractions_per_patient = BED.shape
        self.access_counter = 0
        self._accessed = set()

    def _getBED(self, i, j):
        """
        A class method simulating the actual generation of BED values.
        If the requested value has been already computed it returns without delay, else it
        delays for the time needed to compute it.
        """ 
        if (i,j) not in self._accessed:
            self.access_counter += 1
            self._accessed.add((i,j))
            # replace this by actual time cost, i.e 5*60 seconds
            time.sleep(0)

        return self._BED[i,j]
    
    def root_mean_square_error(self, granularity):
        """
        Computes and returns the mean squared error between the actual and the estimated BED
        matrix.
        """
        estimated = self.estimate(granularity)
        difference = estimated - self._BED
        return np.sqrt((difference ** 2).mean(axis=None))

        
    def estimate(self, granularity):
        """
        Check some points in regural intervals and linearly interpolate the rest.
        
        @param granurality: number of actual measurements to check
        @return estimate, cost: estimated BED matrix, time cost in seconds to compute it
        """
        if(granularity <= 1):
            print("Granularity should be more than 1 silly!")
            return None
  
        self.access_counter = 0
        self._accessed = set()
        
        div = self.max_fractions_per_patient / (granularity - 1) - 1
        
        # Retrieve actual values with the simulated getter - This is time consuming.
        actual_indeces = [int(i*div) for i in range(granularity)] 

        actual_values = [self._getBED(i,j) for i in range(self.num_patients) for j in actual_indeces]
        
        estimate = np.empty(shape = (self.num_patients, self.max_fractions_per_patient)) * np.nan
        # Init with actual measurements  
        estimate[:, actual_indeces] = np.reshape(actual_values, (self.num_patients, len(actual_indeces)))
        # Interpolate the rest linearly
        estimate = pd.DataFrame(estimate).interpolate(axis = 1).values
        
        return estimate

x = BEDPredictor(benefit)
# What is the estimated BED matrix?
estimate = x.estimate(granularity = 3)
# How many accesses were needed to compute that estimation?
num_accessed = x.access_counter
# What is the RMSE of this estimation?
rmse = x.root_mean_square_error(granularity = 3)
            
        

In [ ]:
class ProtonOptimizer(object):
    """
    An abstract class defining a common interface to be used by all our future models.
    Make sure you comply to the API meaning that your concrete class should:
    1) Be constructed by providing the BED values as a 2D np array and the capacity
    2) It should return its result by implementing the get_optimum() method
    """
    def __init__(self, BED, capacity = 100, model_name = 'abstract_therapy'):
        raise NotImplementedError("You cannot construct an abstract class")
    
    def get_optimum():
        """Returns a dictionary (int -> int) from patient ID to fractions"""
        return {}
    
    def pretty_print():
        """Override to print your models output in a meaningful way for debugging purposes"""
        print("The optimum is " + str(self.get_optimum()))

In [ ]:
class LPOptimizer(ProtonOptimizer):
    """Concrete implementation of the ProtonOptimizer interface"""
    def __init__(self, BED, capacity = 100, model_name = 'proton_therapy'):
        self._BED = BED
        num_patients, max_fractions_per_patient = BED.shape
        self.patients = [i for i in range(num_patients)]
        self.fractions = [j for j in range(max_fractions_per_patient)]
        self.m = Model(model_name)
        
        # Set binary decision variables
        self.x = self.m.addVars(num_patients, max_fractions_per_patient, vtype = GRB.BINARY) 
        
        # Only one choice of fractions per patient is valid
        self.m.addConstrs(quicksum(self.x[i,j] for j in self.fractions) == 1 for i in self.patients)
        
        # We can only perform so many proton therapies per week
        self.m.addConstr(quicksum(
            quicksum(self.x[i,j] * self.fractions[j] for j in self.fractions) 
            for i in self.patients) <= capacity)
        self.m.update()
        
        self.optimum = {}

        
    def _solve(self, debug = False):
        # Set objective
        self.m.setObjective(quicksum(
            self.x[i,j] * self._BED[i,j] for i in self.patients for j in self.fractions),
                            GRB.MAXIMIZE)
        
        self.m.setParam('OutputFlag', debug)
        self.m.update()
        self.m.optimize()
        if self.m.status == GRB.Status.OPTIMAL:
            solution = self.m.getAttr("x", self.x)
            for i in self.patients:
                for j in self.fractions:
                    if(solution[i,j] == 1):
                        self.optimum[i] = j
                        break
        else:
            print("Infeasible model")
    
    def pretty_print(self):
        solution = self.get_optimum()
        for patient, fractions in solution.items():
            print(("Patient " + str(patient) + " should receive " + str(fractions) + " fractions"))
        
    def get_optimum(self):
        if not self.optimum:
            self._solve()
        return self.optimum

In [ ]:
import unittest
import random

class ModelingTest(unittest.TestCase):

    def setUp(self):
        """Create a single Proton optimization model."""
        pass
        
    def mock_BED_data(self):
        BED = np.array([[1, 10, 11], [1, 2, 3], [9, 10, 11]])
        max_fractions = BED.shape[1] - 1
        return BED, max_fractions 
    
    def test_infinite_capacity(self):
        """When capacity is infinite, we expect max amount of fractions for all patients"""
        BED, max_fractions = self.mock_BED_data()
        inf_capacity = 10000
        optimizer = LPOptimizer(BED, capacity = inf_capacity)
        for patient, fractions in optimizer.get_optimum().items():
            self.assertEqual(fractions, max_fractions)

    def test_capacity_constraint(self):
        """An optimal model surely uses as many fractions as it can"""
        BED, max_fractions = self.mock_BED_data()
        num_patients = BED.shape[0]
        
        # Capacity should be unable to fulfill every patient
        capacity = random.randint(1, max_fractions * num_patients)
        
        optimizer = LPOptimizer(BED, capacity = capacity)
        solution = optimizer.get_optimum()
        fractions_used = sum(solution.values())
        self.assertEqual(fractions_used, capacity)

    def tearDown(self):
        """Delete all models."""
        pass

# Run tests
a = ModelingTest()
suite = unittest.TestLoader().loadTestsFromModule(a)
unittest.TextTestRunner().run(suite)